In [1]:
from src.agent.powerful import graph


In [3]:
from langchain_google_vertexai import ChatVertexAI

model = ChatVertexAI(model_name="gemini-2.0-flash", project="dev-muhammad-fauzan", location="global")

In [2]:
from src.crud import chat_crud
from uuid_extensions import uuid7
from src.models.helper import from_langchain_message
from src.models.chat_models import ChatMessage, ChatSession

In [3]:
USER_ID = "2"
chat_session = ChatSession(
    id=uuid7(),
    user_id=USER_ID,
    title="Graph Test",
    agent="powerful"
)

In [4]:
chat_session.id

UUID('068346ad-7eb8-72e4-8000-d0775aad5167')

In [4]:
from src.db.database import engine, Session

In [41]:
with Session(engine) as session:
    USER_ID = "2"
    chat_session = ChatSession(
        id=uuid7(),
        user_id=USER_ID,
        title="Graph Test",
        agent="powerful"
    )
    chat_session = chat_crud.create_chat_session(session, chat_session)


In [9]:
from langgraph.graph.state import CompiledStateGraph

In [27]:
from uuid import UUID
from src.crud import chat_crud

def save_response(db: Session, session_id: UUID, responses: list):
    for response in responses:
        for key, value in response.items():
            messages = value.get("messages", [])
            for message in messages:
                chat_message = from_langchain_message(message, session_id=session_id)
                chat_crud.add_chat_message(db=db, chat_message=chat_message)

def process_stream(db: Session, agent: CompiledStateGraph, input: dict, config: dict, chat_session: ChatSession):
    langchain_messages = input.get("messages", [])
    events = []
    run = {}
    event_messages = []
    for mode, event in agent.stream(input=input, config=config, 
        stream_mode=["updates", "messages"]
    ):
        events.append(event)
        if mode == "updates":
            for key, value in event.items():
                messages = value.get("messages", [])
                if isinstance(messages, list):
                    langchain_messages.extend(messages)
                    for message in messages:
                        chat_message = from_langchain_message(message, chat_session.id)
                        if message.id in run:
                            run_message = run[message.id]
                            # print(chat_message)
                            run_message.content = chat_message.content
                            run_message.role = chat_message.role
                            run_message.tool_call_id = chat_message.tool_call_id
                            run_message.tool_calls = chat_message.tool_calls
                            run_message.additional_kwargs = chat_message.additional_kwargs
                            # chat_message = run_message
                            chat_message = chat_crud.add_chat_message(db, run_message)
                            run[message.id] = chat_message
                            # print("update message", run_message)
                        else:
                            # chat_message = from_langchain_message(message, chat_session.id)
                            chat_message = chat_crud.add_chat_message(db, chat_message)
                        yield {"mode": "messages", "message": chat_message}
                        
                else:
                    langchain_messages.append(*messages)

            
        elif mode == "messages":
            message = event[0]
            chat_message = from_langchain_message(message, chat_session.id)
            # print(message.id, "message", message)
            # print(message.id, "chat_message", chat_message)
            if message.id in run:
                run[message.id].content = run[message.id].content + message.content
                # print("update run message", run[message.id])
                chat_message = chat_crud.update_message_content_by_id(db, message.id, run[message.id].content)
            else:
                run[message.id] = chat_message
                chat_message = chat_crud.add_chat_message(db, chat_message)
            event_messages.append(event)

            yield {"mode": "messages", "message": chat_message}

def chat_agent(*, db: Session, message: str, session_id: UUID, user_id: str):
    chat_session = chat_crud.get_chat_session_by_id(db=db, session_id=session_id)
    if not chat_session:
        raise ValueError("Chat session not found")
    

    chat_messages = chat_crud.get_chat_messages_by_session_id(db=db, session_id=session_id)
    chat_messages = [m.to_langchain_message() for m in chat_messages]

    chat_message = ChatMessage(
        id=str(uuid7()),
        session_id=session_id,
        role="human",
        content=message
    )
    chat_crud.add_chat_message(db=db, chat_message=chat_message)

    chat_messages.append(chat_message.to_langchain_message())

    config = {
        "configurable": {
            "user_id": user_id,
            "session_id": str(session_id),
        }
    }

    input = {
        "messages": chat_messages
    }

    for event in process_stream(db=db, agent=graph, input=input, config=config, chat_session=chat_session):
        yield event
    # save_response(db=db, session_id=session_id, responses=response)
    # return response

In [11]:
with Session(engine) as session:
    response = chat_agent(
        db=session,
        message="""hi""",
        session_id=chat_session.id,
        user_id=USER_ID
    )

In [13]:
for e in response:
    print(e)

2025/05/26 21:05:14 WARNING mlflow.tracing.processor.mlflow: Creating a trace within the default experiment with id '0'. It is strongly recommended to not use the default experiment to log traces due to ambiguous search results and probable performance issues over time due to directory table listing performance degradation with high volumes of directories within a specific path. To avoid performance and disambiguation issues, set the experiment for your environment using `mlflow.set_experiment()` API.


{'mode': 'messages', 'message': ChatMessage(role=<MessageRole.AI: 'ai'>, session_id=UUID('0683471f-5d96-728c-8000-e4a39f5ea499'), message_id=UUID('06834751-cac6-7fbd-8000-00216195c40c'), tool_calls=[], additional_kwargs={}, updated_at=datetime.datetime(2025, 5, 26, 21, 5, 16, 690589), deleted_at=None, id='run--b52bb8e6-49eb-4c51-bc8e-2997253a8824', content='Hi', tool_call_id=None, created_at=datetime.datetime(2025, 5, 26, 21, 5, 16, 690148))}
{'mode': 'messages', 'message': ChatMessage(role=<MessageRole.AI: 'ai'>, session_id=UUID('0683471f-5d96-728c-8000-e4a39f5ea499'), message_id=UUID('06834751-cac6-7fbd-8000-00216195c40c'), tool_calls=[], additional_kwargs={}, updated_at=datetime.datetime(2025, 5, 26, 21, 5, 16, 690589), deleted_at=None, id='run--b52bb8e6-49eb-4c51-bc8e-2997253a8824', content='Hi, how can I help you with your investment analysis today? Are you interested in analyzing a specific', tool_call_id=None, created_at=datetime.datetime(2025, 5, 26, 21, 5, 16, 690148))}
{'mode

In [18]:
message = e["message"]

In [22]:
message.model_dump_json()

'{"role":"ai","session_id":"0683471f-5d96-728c-8000-e4a39f5ea499","message_id":"06834751-cac6-7fbd-8000-00216195c40c","tool_calls":[],"additional_kwargs":{},"updated_at":"2025-05-26T21:05:16.690589","deleted_at":null,"id":"run--b52bb8e6-49eb-4c51-bc8e-2997253a8824","content":"Hi, how can I help you with your investment analysis today? Are you interested in analyzing a specific stock, getting insights on your portfolio, or something else?\\n","tool_call_id":null,"created_at":"2025-05-26T21:05:16.690148"}'

In [42]:
with Session(engine) as session:
    save_response(
        db=session,
        session_id=chat_session.id,
        responses=response
    )

In [18]:
with Session(engine) as session:
    chat_messages = chat_crud.get_chat_messages_by_session_id(db=session, session_id=chat_session.id)

In [28]:
chat_session.id

UUID('068346d1-1b94-7877-8000-0917e3e9f1f0')

In [32]:
chat_session.id

UUID('0683471f-5d96-728c-8000-e4a39f5ea499')

In [42]:
agent = graph
from langchain_core.messages import HumanMessage


user_input = """Cool, I got this headline news Iran Tolak Setop Pengayaan Uranium Demi Kesepakatan dengan AS"""
new_input = HumanMessage(user_input)
human = from_langchain_message(new_input, chat_session.id)
with Session(engine) as session:
    chat_crud.add_chat_message(session, human)

In [ ]:
# from src.crud import chat_crud
# with Session(engine) as session:
#     chat_session = chat_crud.get_chat_session_by_id(session, '06834690-13dd-7edf-8000-18e47b4f6e66')

In [43]:
langchain_messages = [new_input]

config = {
    "configurable": {
        "user_id": "2"
    }
}

events = []
event_messages = []
run = {}
agent = graph
with Session(engine) as session:
    async for mode, event in agent.astream({"messages":langchain_messages}, config=config, 
        stream_mode=["updates", "messages"]
    ):
        events.append(event)
        if mode == "updates":
            for key, value in event.items():
                messages = value.get("messages", [])
                if isinstance(messages, list):
                    langchain_messages.extend(messages)
                    for message in messages:
                        if message.id in run:
                            run_message = run[message.id]
                            chat_message = from_langchain_message(message, chat_session.id)
                            # print(chat_message)
                            run_message.content = chat_message.content
                            run_message.role = chat_message.role
                            run_message.tool_call_id = chat_message.tool_call_id
                            run_message.tool_calls = chat_message.tool_calls
                            run_message.additional_kwargs = chat_message.additional_kwargs
                            # chat_message = run_message
                            chat_message = chat_crud.add_chat_message(session, run_message)
                            run[message.id] = chat_message
                            print("update message", run_message)
                        else:
                            chat_message = from_langchain_message(message, chat_session.id)
                            chat_message = chat_crud.add_chat_message(session, chat_message)
                        
                else:
                    langchain_messages.append(*messages)
        elif mode == "messages":
            message = event[0]
            chat_message = from_langchain_message(message, chat_session.id)
            print(message.id, "message", message)
            print(message.id, "chat_message", chat_message)
            if message.id in run:
                run[message.id].content = run[message.id].content + message.content
                print("update run message", run[message.id])
                chat_message = chat_crud.update_message_content_by_id(session, message.id, run[message.id].content)
            else:
                run[message.id] = chat_message
                chat_message = chat_crud.add_chat_message(session, chat_message)
            event_messages.append(event)

run--3cb5ba53-e266-4add-bc6c-8227e03fc2d4 message content='Okay, I will' additional_kwargs={} response_metadata={'safety_ratings': [], 'usage_metadata': {}} id='run--3cb5ba53-e266-4add-bc6c-8227e03fc2d4'
run--3cb5ba53-e266-4add-bc6c-8227e03fc2d4 chat_message session_id=UUID('06834834-8393-7b6c-8000-b3d3b42d8c87') id='run--3cb5ba53-e266-4add-bc6c-8227e03fc2d4' role=<MessageRole.AI: 'ai'> content='Okay, I will' tool_calls=[] additional_kwargs={} message_id=UUID('06834836-944e-7571-8000-0605a085c6f7') tool_call_id=None created_at=datetime.datetime(2025, 5, 26, 22, 6, 17, 269539) updated_at=datetime.datetime(2025, 5, 26, 22, 6, 17, 269730) deleted_at=None
run--3cb5ba53-e266-4add-bc6c-8227e03fc2d4 message content=' analyze the sentiment of the headline you provided. Since the headline is in Indonesian, I will' additional_kwargs={} response_metadata={'safety_ratings': [], 'usage_metadata': {}} id='run--3cb5ba53-e266-4add-bc6c-8227e03fc2d4'
run--3cb5ba53-e266-4add-bc6c-8227e03fc2d4 chat_messa

I0000 00:00:1748271978.081716 1661332 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


run--f6a2b360-122c-4bbc-8fa1-d94eb74af6dd message content='Iran' additional_kwargs={} response_metadata={'safety_ratings': [], 'usage_metadata': {}} id='run--f6a2b360-122c-4bbc-8fa1-d94eb74af6dd'
run--f6a2b360-122c-4bbc-8fa1-d94eb74af6dd chat_message session_id=UUID('06834834-8393-7b6c-8000-b3d3b42d8c87') id='run--f6a2b360-122c-4bbc-8fa1-d94eb74af6dd' role=<MessageRole.AI: 'ai'> content='Iran' tool_calls=[] additional_kwargs={} message_id=UUID('06834836-c6d4-74cb-8000-4f5c506c4775') tool_call_id=None created_at=datetime.datetime(2025, 5, 26, 22, 6, 20, 427612) updated_at=datetime.datetime(2025, 5, 26, 22, 6, 20, 427985) deleted_at=None
run--f6a2b360-122c-4bbc-8fa1-d94eb74af6dd message content=' Re' additional_kwargs={} response_metadata={'safety_ratings': [], 'usage_metadata': {}} id='run--f6a2b360-122c-4bbc-8fa1-d94eb74af6dd'
run--f6a2b360-122c-4bbc-8fa1-d94eb74af6dd chat_message session_id=UUID('06834834-8393-7b6c-8000-b3d3b42d8c87') id='run--f6a2b360-122c-4bbc-8fa1-d94eb74af6dd' rol

run--1b26635a-a68d-4e5c-b806-8e8d7d8bc283 message content='' additional_kwargs={'function_call': {'name': 'analyze_sentiment', 'arguments': '{"text": "Iran Rejects Stopping Uranium Enrichment for the Sake of a Deal with the US"}'}} response_metadata={'safety_ratings': [], 'usage_metadata': {}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash'} id='run--1b26635a-a68d-4e5c-b806-8e8d7d8bc283' tool_calls=[{'name': 'analyze_sentiment', 'args': {'text': 'Iran Rejects Stopping Uranium Enrichment for the Sake of a Deal with the US'}, 'id': '48165c07-7c3d-415b-9024-a78302f14182', 'type': 'tool_call'}] usage_metadata={'input_tokens': 3502, 'output_tokens': 53, 'total_tokens': 3555} tool_call_chunks=[{'name': 'analyze_sentiment', 'args': '{"text": "Iran Rejects Stopping Uranium Enrichment for the Sake of a Deal with the US"}', 'id': '48165c07-7c3d-415b-9024-a78302f14182', 'index': None, 'type': 'tool_call_chunk'}]
run--1b26635a-a68d-4e5c-b806-8e8d7d8bc283 chat_message session_id=UUID('068

In [8]:
from langchain_core.messages import HumanMessage
hm = HumanMessage("Thank you")
followup = langchain_messages + [hm]
r = agent.invoke(
    {"messages": followup},
    config=config
)
r

NameError: name 'langchain_messages' is not defined

In [45]:
chat_session.id

UUID('06834834-8393-7b6c-8000-b3d3b42d8c87')

In [6]:
with Session(engine) as session:
    a = [m.to_langchain_message() for m in chat_crud.get_chat_messages_by_session_id(session, '06834834-8393-7b6c-8000-b3d3b42d8c87')]
a

[HumanMessage(content='Cool, I got this headline news Iran Tolak Setop Pengayaan Uranium Demi Kesepakatan dengan AS', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Okay, I will analyze the sentiment of the headline you provided. Since the headline is in Indonesian, I will first translate it to English. Then I will analyze the sentiment of the translated text.\n', additional_kwargs={'function_call': {'name': 'translate_to_english', 'arguments': '{"text": "Iran Tolak Setop Pengayaan Uranium Demi Kesepakatan dengan AS"}'}}, response_metadata={}, tool_calls=[{'name': 'translate_to_english', 'args': {'text': 'Iran Tolak Setop Pengayaan Uranium Demi Kesepakatan dengan AS'}, 'id': 'ea19bd7d-edcf-4aec-a622-9755c6899149', 'type': 'tool_call'}]),
 AIMessage(content='Iran Rejects Stopping Uranium Enrichment for the Sake of a Deal with the US\n', additional_kwargs={}, response_metadata={}),
 ToolMessage(content='{"status": "success", "data": {"translated_text": "Iran Rejects Sto

In [10]:
langchain_messages

[HumanMessage(content='I got news "China, Indonesia Sovereign Wealth Funds Boost Investment Ties", evaluate my portofolio in finance sector', additional_kwargs={}, response_metadata={}, id='974dc75f-56f9-4e55-ae59-943440915265'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_user_portofolio_company', 'arguments': '{}'}}, response_metadata={'safety_ratings': [], 'usage_metadata': {}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash'}, id='run--0a412bae-0fdd-4bfd-aeb3-3ab93f6e2960', tool_calls=[{'name': 'get_user_portofolio_company', 'args': {}, 'id': '398d5cd6-cb6d-406e-8821-91946deaaf09', 'type': 'tool_call'}], usage_metadata={'input_tokens': 3418, 'output_tokens': 8, 'total_tokens': 3426}),
 ToolMessage(content='{"status": "success", "data": [{"company_name": "PT Aneka Tambang Tbk", "ticker": "ANTM"}, {"company_name": "PT Astra Otoparts Tbk", "ticker": "AUTO"}, {"company_name": "PT Avia Avian Tbk", "ticker": "AVIA"}, {"company_name": "PT Bank Central

In [25]:
for i in range(len(event_messages)):
    print(event_messages[i][0].id)

run--553d61f2-f560-4402-83d0-6f8013e21ea2
run--3a61e201-64d1-40ad-abd4-5f3f4d72f380
run--3a61e201-64d1-40ad-abd4-5f3f4d72f380
run--3a61e201-64d1-40ad-abd4-5f3f4d72f380
75d95107-1a73-40b6-bee8-bd694340c43a
run--544e3085-7147-4bd7-8295-ca523d496473
84478e02-c083-44ea-9be1-ce0895b91c2b
run--9b9b82ad-b2bd-4a71-b3da-e143b521a402
run--9b9b82ad-b2bd-4a71-b3da-e143b521a402
run--9b9b82ad-b2bd-4a71-b3da-e143b521a402
run--9b9b82ad-b2bd-4a71-b3da-e143b521a402


In [13]:
session_id = chat_session.id
app_messages = [from_langchain_message(m, session_id=session_id) for m in langchain_messages]

app_messages

[ChatMessage(session_id=UUID('06834285-2ff7-7fae-8000-b6887f57c5cd'), id='d897f970-18ee-4d10-a924-5d0f64bfb9e9', role=<MessageRole.HUMAN: 'human'>, content='I got news "China, Indonesia Sovereign Wealth Funds Boost Investment Ties", evaluate my portofolio in finance sector', additional_kwargs={}, message_id=UUID('0683428b-ecf4-7395-8000-dd96a97343fc'), tool_calls=[], tool_call_id=None, created_at=datetime.datetime(2025, 5, 26, 15, 39, 26, 824960), updated_at=datetime.datetime(2025, 5, 26, 15, 39, 26, 825264), deleted_at=None),
 ChatMessage(session_id=UUID('06834285-2ff7-7fae-8000-b6887f57c5cd'), id='run--89854fa4-48dd-4ad1-915d-abe640d4393b-0', role=<MessageRole.AI: 'ai'>, content='', tool_calls=[{'id': '74a897bd-21b5-4ce4-ab78-9f1fbe691dde', 'name': 'get_user_portofolio_company', 'args': {}, 'type': 'tool_call'}], additional_kwargs={'function_call': {'name': 'get_user_portofolio_company', 'arguments': '{}'}}, message_id=UUID('0683428b-ed35-7829-8000-9938fd18fc8a'), tool_call_id=None, 

In [10]:
app_messages[1].model_dump()

{'session_id': UUID('06834285-2ff7-7fae-8000-b6887f57c5cd'),
 'id': 'run--89854fa4-48dd-4ad1-915d-abe640d4393b-0',
 'role': <MessageRole.AI: 'ai'>,
 'content': '',
 'tool_calls': [{'id': '74a897bd-21b5-4ce4-ab78-9f1fbe691dde',
   'name': 'get_user_portofolio_company',
   'args': {},
   'type': 'tool_call'}],
 'additional_kwargs': {'function_call': {'name': 'get_user_portofolio_company',
   'arguments': '{}'}},
 'message_id': UUID('06834288-1704-7c27-8000-49eb7f015603'),
 'tool_call_id': None,
 'created_at': datetime.datetime(2025, 5, 26, 15, 38, 25, 439164),
 'updated_at': datetime.datetime(2025, 5, 26, 15, 38, 25, 439375),
 'deleted_at': None}

In [11]:
from typing import TypedDict

In [12]:
import json
from src.models.chat_models import ToolCall

json.dumps([ToolCall(id="123", name="test", args={"key": "value"}, type="function")], indent=2)

'[\n  {\n    "id": "123",\n    "name": "test",\n    "args": {\n      "key": "value"\n    },\n    "type": "function"\n  }\n]'

In [14]:
print(app_messages[1].model_dump())
# app_messages[2].tool_calls = [ToolCall(id= "a", name="evaluate_portfolio", args={"portfolio": "finance sector"}, type="function")]
with Session(engine) as session:
    chat_crud.add_chat_message(session, app_messages[1])

{'session_id': UUID('06834285-2ff7-7fae-8000-b6887f57c5cd'), 'id': 'run--89854fa4-48dd-4ad1-915d-abe640d4393b-0', 'role': <MessageRole.AI: 'ai'>, 'content': '', 'tool_calls': [{'id': '74a897bd-21b5-4ce4-ab78-9f1fbe691dde', 'name': 'get_user_portofolio_company', 'args': {}, 'type': 'tool_call'}], 'additional_kwargs': {'function_call': {'name': 'get_user_portofolio_company', 'arguments': '{}'}}, 'message_id': UUID('0683428b-ed35-7829-8000-9938fd18fc8a'), 'tool_call_id': None, 'created_at': datetime.datetime(2025, 5, 26, 15, 39, 26, 825805), 'updated_at': datetime.datetime(2025, 5, 26, 15, 39, 26, 826021), 'deleted_at': None}


In [72]:
langchain_messages[1]

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_user_portofolio_company', 'arguments': '{}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 3418, 'candidates_token_count': 8, 'total_token_count': 3426, 'prompt_tokens_details': [{'modality': 1, 'token_count': 3418}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 8}], 'thoughts_token_count': 0, 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.005013489630073309, 'model_name': 'gemini-2.0-flash'}, id='run--825a6b63-13da-4059-8428-44f8d06b01d6-0', tool_calls=[{'name': 'get_user_portofolio_company', 'args': {}, 'id': '7f5cbb8b-6f1c-436f-9d51-37b3c8060caf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 3418, 'output_tokens': 8, 'total_tokens': 3426})

In [43]:
r_m = []
for app_message in app_messages:
    r_m.append(app_message.to_langchain_message())

In [52]:
agent.get_prompts(config=config)

[]

In [38]:
a = ["a"]
a.extend(["b"])
a

['a', 'b']

In [ ]:
for key, value in events[4]['Sentiment Analyst Agent'].items():
    print(key)

messages


In [ ]:
for message in events[4]['Sentiment Analyst Agent']['messages']:
    print(from_langchain_message(message, uuid7()))

session_id=UUID('068332b2-e8c5-78dd-8000-12c365a9dc5e') id='run--c85b9f3f-10ff-42c3-a619-347c6f5b32c2-0' role=<MessageRole.AI: 'ai'> content="Neutral sentiment detected (0.0) in translated news: <![TRANSLATED]!> 'How is the news sentiment with \\'Secretary-General of ASEAN discusses key regional issues with The Star Media\\''\n" tool_calls=[] additional_kwargs={} message_id=UUID('068332b2-e8c8-7861-8000-d1b41904a60e') tool_call_id=None created_at=datetime.datetime(2025, 5, 25, 21, 37, 34, 572703) updated_at=datetime.datetime(2025, 5, 25, 21, 37, 34, 573055) deleted_at=None
session_id=UUID('068332b2-e92d-77d7-8000-542c0e259d0d') id='7324c193-7ee8-45e8-aafc-fcee8f88e050' role=<MessageRole.AI: 'ai'> content='Transferring back to supervisor' tool_calls=[ToolCall(id='0ee4738b-a80c-459f-b661-5d3b395b43d1', name='transfer_back_to_supervisor', args={}, type='tool_call')] additional_kwargs={} message_id=UUID('068332b2-e92e-7275-8000-16dea9c8d404') tool_call_id=None created_at=datetime.datetime(

In [1]:
from src.agent.powerful import graph
from src.app.agent_process import Agent
from src.db.database import engine
from sqlmodel import Session
from uuid_extensions import uuid7

In [2]:
session_id = str(uuid7())
config = {
    "configurable": {
        "user_id": 2
    }
}
with Session(engine) as session:
    agent = Agent(graph=graph, session_id=session_id, db_session=session, user_id=2)
    
    for e in agent.run(message="Apa saja stock yang ada di portofolio saya?", config=config):
        print(e)

2025/05/27 23:01:42 WARNING mlflow.tracing.processor.mlflow: Creating a trace within the default experiment with id '0'. It is strongly recommended to not use the default experiment to log traces due to ambiguous search results and probable performance issues over time due to directory table listing performance degradation with high volumes of directories within a specific path. To avoid performance and disambiguation issues, set the experiment for your environment using `mlflow.set_experiment()` API.


event: start
data: {"session_id": "06835e1e-60ff-7c43-8000-c63acdbe9ac6", "user_id": 2, "agent": "LangGraph", "title": ""}




event: message-delta
data: {"message": {"content": "", "additional_kwargs": {"function_call": {"name": "get_user_portofolio_company", "arguments": "{}"}}, "response_metadata": {"safety_ratings": [], "usage_metadata": {}, "finish_reason": "STOP", "model_name": "gemini-2.0-flash"}, "type": "AIMessageChunk", "name": null, "id": "run--225280c6-8c39-463b-84c9-5ab059ee88f9", "example": false, "tool_calls": [{"name": "get_user_portofolio_company", "args": {}, "id": "c8889f02-abde-419a-81ad-513f11cc2f24", "type": "tool_call"}], "invalid_tool_calls": [], "usage_metadata": {"input_tokens": 3407, "output_tokens": 8, "total_tokens": 3415}, "tool_call_chunks": [{"name": "get_user_portofolio_company", "args": "{}", "id": "c8889f02-abde-419a-81ad-513f11cc2f24", "index": null, "type": "tool_call_chunk"}]}, "langgraph_info": {"user_id": 2, "langgraph_step": 1, "langgraph_node": "agent", "langgraph_triggers": ["branch:to:agent"], "langgraph_path": ["__pregel_pull", "agent"], "langgraph_checkpoint_ns": "

In [17]:
a = {"b": "c", "d": "e"}

In [20]:
f = a.get("f", {})
f.update({"g":"h"})

In [21]:
f

{'g': 'h'}